# Vector Databases: FAISS and ChromaDB

This notebook covers Vector Databases for efficient similarity search:
- **What are Vector DBs?**: Specialized databases for storing and searching high-dimensional vectors
- **FAISS**: Facebook AI Similarity Search - fast similarity search library
- **ChromaDB**: Open-source embedding database with built-in features
- **Use Cases**: RAG, semantic search, recommendation systems, and more
- **Practical Examples**: Building and querying vector databases

## Learning Objectives

- Understand what vector databases are and why they're needed
- Learn to use FAISS for efficient similarity search
- Learn to use ChromaDB for embedding storage and retrieval
- Compare different vector database approaches
- Apply vector DBs for real-world scenarios


## Installation

Run this cell to install required packages (uncomment if needed):


In [ ]:
# Install packages (uncomment if needed)
# !pip install faiss-cpu chromadb sentence-transformers numpy pandas


## 1. What are Vector Databases?

**Vector Databases** are specialized databases designed to store and efficiently search high-dimensional vectors (embeddings). Unlike traditional databases that search by exact matches, vector DBs enable:

- **Similarity Search**: Find vectors similar to a query vector
- **Scalability**: Handle millions or billions of vectors efficiently
- **Speed**: Optimized algorithms for fast nearest neighbor search
- **Metadata Storage**: Store additional information alongside vectors

### Why Vector DBs?

When you have thousands or millions of embeddings, calculating similarity with all vectors becomes slow. Vector DBs use:
- **Indexing**: Special data structures (IVF, HNSW, etc.) for fast search
- **Approximate Search**: Trade some accuracy for significant speed gains
- **GPU Support**: Leverage GPU acceleration for faster searches

### Common Use Cases:
- **RAG (Retrieval Augmented Generation)**: Find relevant documents for LLM context
- **Semantic Search**: Search by meaning, not keywords
- **Recommendation Systems**: Find similar items/users
- **Deduplication**: Find duplicate or similar content


In [1]:
# Import libraries
from sentence_transformers import SentenceTransformer
import os
import time

print("Libraries imported successfully!")


Libraries imported successfully!


## 2. FAISS (Facebook AI Similarity Search)

**FAISS** is a library developed by Facebook AI Research for efficient similarity search and clustering of dense vectors. It's:

- **Fast**: Optimized C++ implementation with Python bindings
- **Scalable**: Handles billions of vectors
- **Flexible**: Multiple indexing methods (exact, approximate)
- **Lightweight**: No external dependencies for basic usage

### Key Features:
- Multiple index types (Flat, IVF, HNSW, etc.)
- GPU support
- Batch operations
- Memory-efficient


In [2]:
# Load embedding model
try:
    model = SentenceTransformer('all-MiniLM-L6-v2')
    print(f"Model loaded: {model.get_sentence_embedding_dimension()} dimensions")
except Exception as e:
    print(f"Error loading model: {e}")
    model = None


Model loaded: 384 dimensions


In [3]:
try:
    import faiss
    
    # Sample documents
    documents = [
        "Machine learning is a subset of artificial intelligence",
        "Deep learning uses neural networks with multiple layers",
        "Natural language processing helps computers understand human language",
        "Computer vision enables machines to interpret visual information",
        "Reinforcement learning trains agents through rewards and penalties",
        "Supervised learning uses labeled data to train models",
        "Unsupervised learning finds patterns in unlabeled data",
        "I love eating pizza on weekends",
        "The weather is beautiful today",
        "Python is a popular programming language for data science"
    ]
    
    if model:
        # Generate embeddings
        print("Generating embeddings...")
        embeddings = model.encode(documents, show_progress_bar=True)
        dimension = embeddings.shape[1]
        print(f"Generated {len(documents)} embeddings of dimension {dimension}")
        
        # Normalize embeddings for cosine similarity (FAISS uses L2 distance)
        # For cosine similarity, we normalize vectors
        faiss.normalize_L2(embeddings)
        
        # Create FAISS index
        # IndexFlatL2: Exact search using L2 (Euclidean) distance
        # For normalized vectors, L2 distance ≈ cosine distance
        index = faiss.IndexFlatL2(dimension)
        
        # Add vectors to index
        index.add(embeddings.astype('float32'))
        
        print(f"\nFAISS index created with {index.ntotal} vectors")
        print(f"Index type: {type(index).__name__}")
        
    else:
        print("Model not available")
        index = None
        embeddings = None
        
except ImportError:
    print("FAISS not installed. Install with: pip install faiss-cpu")
    index = None
    embeddings = None
    documents = []


Generating embeddings...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated 10 embeddings of dimension 384

FAISS index created with 10 vectors
Index type: IndexFlatL2


### 2.1 FAISS - Basic Example with Flat Index


In [4]:
# Search in FAISS index
if index is not None and model is not None:
    # Query
    query = "How do neural networks learn?"
    
    # Generate query embedding
    query_embedding = model.encode([query])
    faiss.normalize_L2(query_embedding)
    query_embedding = query_embedding.astype('float32')
    
    # Search for top 3 similar vectors
    k = 3
    distances, indices = index.search(query_embedding, k)
    
    print(f"Query: '{query}'\n")
    print("Top 3 Most Similar Documents:")
    print("-" * 70)
    
    for i, (idx, dist) in enumerate(zip(indices[0], distances[0]), 1):
        print(f"\nRank {i} (Distance: {dist:.4f}):")
        print(f"  {documents[idx]}")
        
else:
    print("Index not available")


Query: 'How do neural networks learn?'

Top 3 Most Similar Documents:
----------------------------------------------------------------------

Rank 1 (Distance: 0.9683):
  Deep learning uses neural networks with multiple layers

Rank 2 (Distance: 1.1865):
  Machine learning is a subset of artificial intelligence

Rank 3 (Distance: 1.2397):
  Supervised learning uses labeled data to train models


### 2.2 FAISS - Approximate Search with IVF Index

For larger datasets, exact search becomes slow. FAISS provides approximate search methods that trade some accuracy for significant speed improvements.


In [5]:
if embeddings is not None:
    try:
        dimension = embeddings.shape[1]
        n_vectors = len(embeddings)
        
        # Create IVF (Inverted File Index) for approximate search
        # nlist: number of clusters (centroids)
        nlist = min(4, n_vectors // 2)  # Use 4 clusters for small dataset
        
        # Quantizer for clustering
        quantizer = faiss.IndexFlatL2(dimension)
        
        # IVF index
        index_ivf = faiss.IndexIVFFlat(quantizer, dimension, nlist)
        
        # Train the index (required for IVF)
        print("Training IVF index...")
        index_ivf.train(embeddings.astype('float32'))
        
        # Add vectors
        index_ivf.add(embeddings.astype('float32'))
        
        # Set number of probes (how many clusters to search)
        index_ivf.nprobe = 2
        
        print(f"IVF Index created with {index_ivf.ntotal} vectors")
        print(f"Number of clusters (nlist): {nlist}")
        print(f"Number of probes: {index_ivf.nprobe}")
        
        # Compare search speed
        if model:
            query_embedding = model.encode(["neural networks and deep learning"])
            faiss.normalize_L2(query_embedding)
            query_embedding = query_embedding.astype('float32')
            
            # Time exact search
            start = time.time()
            distances_exact, indices_exact = index.search(query_embedding, 3)
            time_exact = time.time() - start
            
            # Time approximate search
            start = time.time()
            distances_approx, indices_approx = index_ivf.search(query_embedding, 3)
            time_approx = time.time() - start
            
            print(f"\nSearch Performance (for {n_vectors} vectors):")
            print(f"Exact search (Flat): {time_exact*1000:.3f} ms")
            print(f"Approximate search (IVF): {time_approx*1000:.3f} ms")
            
    except Exception as e:
        print(f"Error creating IVF index: {e}")
        print("Note: IVF is more useful for larger datasets (thousands+ vectors)")
else:
    print("Embeddings not available")


Training IVF index...
IVF Index created with 10 vectors
Number of clusters (nlist): 4
Number of probes: 2


WARNING clustering 10 points to 4 centroids: please provide at least 156 training points



Search Performance (for 10 vectors):
Exact search (Flat): 0.538 ms
Approximate search (IVF): 0.481 ms


### 2.3 FAISS - Saving and Loading Index

FAISS allows you to save indexes to disk for persistence.


In [6]:
if index is not None:
    try:
        # Save index to disk
        index_path = "faiss_index.bin"
        faiss.write_index(index, index_path)
        print(f"Index saved to {index_path}")
        
        # Load index from disk
        loaded_index = faiss.read_index(index_path)
        print(f"Index loaded: {loaded_index.ntotal} vectors")
        
        # Verify it works
        if model:
            query_embedding = model.encode(["artificial intelligence"])
            faiss.normalize_L2(query_embedding)
            query_embedding = query_embedding.astype('float32')
            
            distances, indices = loaded_index.search(query_embedding, 2)
            print(f"\nSearch test with loaded index:")
            for i, idx in enumerate(indices[0], 1):
                print(f"  {i}. {documents[idx]}")
        
        # Clean up
        if os.path.exists(index_path):
            os.remove(index_path)
            print(f"\nCleaned up {index_path}")
            
    except Exception as e:
        print(f"Error saving/loading index: {e}")
else:
    print("Index not available")


Index saved to faiss_index.bin
Index loaded: 10 vectors

Search test with loaded index:
  1. Machine learning is a subset of artificial intelligence
  2. Natural language processing helps computers understand human language

Cleaned up faiss_index.bin


## 3. ChromaDB

**ChromaDB** is an open-source embedding database that provides:
- **Easy to use**: Simple Python API
- **Built-in features**: Automatic embedding generation, metadata filtering
- **Persistence**: Save collections to disk
- **Production-ready**: Designed for real applications

### Key Features:
- Automatic embedding generation (or use your own)
- Metadata filtering and querying
- Collection management
- Persistent storage


### 3.1 ChromaDB - Basic Example


In [7]:
try:
    import chromadb
    from chromadb.config import Settings
    
    # Initialize ChromaDB client
    # Using in-memory mode for this example
    # For persistence: client = chromadb.Client()
    client = chromadb.Client(Settings(anonymized_telemetry=False))
    
    # Create or get a collection
    # ChromaDB can automatically generate embeddings, but we'll use our own
    collection = client.create_collection(
        name="ai_documents",
        metadata={"description": "AI and ML related documents"}
    )
    
    print("ChromaDB collection created successfully!")
    
except ImportError:
    print("ChromaDB not installed. Install with: pip install chromadb")
    client = None
    collection = None
except Exception as e:
    print(f"Error initializing ChromaDB: {e}")
    client = None
    collection = None


ChromaDB collection created successfully!


In [8]:
# Add documents to ChromaDB
if collection is not None and model is not None:
    # Generate embeddings
    embeddings_list = model.encode(documents).tolist()
    
    # Add documents with embeddings and metadata
    collection.add(
        embeddings=embeddings_list,
        documents=documents,
        ids=[f"doc_{i}" for i in range(len(documents))],
        metadatas=[
            {"category": "AI" if i < 7 else "other", "index": i}
            for i in range(len(documents))
        ]
    )
    
    print(f"Added {len(documents)} documents to ChromaDB collection")
    print(f"Collection count: {collection.count()}")
else:
    print("Collection or model not available")


Added 10 documents to ChromaDB collection
Collection count: 10


In [9]:
# Query ChromaDB
if collection is not None and model is not None:
    query = "How do neural networks learn?"
    
    # Generate query embedding
    query_embedding = model.encode([query]).tolist()
    
    # Search
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=3
    )
    
    print(f"Query: '{query}'\n")
    print("Top 3 Most Similar Documents:")
    print("-" * 70)
    
    for i, (doc, metadata, doc_id) in enumerate(
        zip(results['documents'][0], results['metadatas'][0], results['ids'][0]), 
        1
    ):
        print(f"\nRank {i} (ID: {doc_id}):")
        print(f"  {doc}")
        print(f"  Metadata: {metadata}")
        
else:
    print("Collection or model not available")


Query: 'How do neural networks learn?'

Top 3 Most Similar Documents:
----------------------------------------------------------------------

Rank 1 (ID: doc_1):
  Deep learning uses neural networks with multiple layers
  Metadata: {'index': 1, 'category': 'AI'}

Rank 2 (ID: doc_0):
  Machine learning is a subset of artificial intelligence
  Metadata: {'category': 'AI', 'index': 0}

Rank 3 (ID: doc_5):
  Supervised learning uses labeled data to train models
  Metadata: {'category': 'AI', 'index': 5}


### 3.2 ChromaDB - Using Built-in Embeddings

ChromaDB can automatically generate embeddings using default models, making it even easier to use.


In [10]:
if client is not None:
    try:
        # Create collection with default embedding function
        # ChromaDB uses sentence-transformers by default
        collection_auto = client.create_collection(
            name="auto_embeddings",
            metadata={"description": "Collection with automatic embeddings"}
        )
        
        # Add documents without providing embeddings
        # ChromaDB will generate them automatically
        collection_auto.add(
            documents=documents[:5],  # Use first 5 documents
            ids=[f"auto_doc_{i}" for i in range(5)],
            metadatas=[
                {"category": "AI", "index": i}
                for i in range(5)
            ]
        )
        
        print(f"Added {collection_auto.count()} documents with auto-generated embeddings")
        
        # Query using text directly (no need to generate embeddings)
        results = collection_auto.query(
            query_texts=["neural networks and machine learning"],
            n_results=2
        )
        
        print("\nQuery Results (using text query):")
        print("-" * 70)
        for i, doc in enumerate(results['documents'][0], 1):
            print(f"{i}. {doc}")
            
    except Exception as e:
        print(f"Error: {e}")
else:
    print("ChromaDB client not available")


/Users/mx98/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 33.5MiB/s]


Added 5 documents with auto-generated embeddings

Query Results (using text query):
----------------------------------------------------------------------
1. Machine learning is a subset of artificial intelligence
2. Deep learning uses neural networks with multiple layers


### 3.3 ChromaDB - Metadata Filtering

One of ChromaDB's powerful features is filtering by metadata.


In [11]:
if collection is not None and model is not None:
    query_embedding = model.encode(["machine learning techniques"]).tolist()
    
    # Query with metadata filter - only AI category documents
    results_filtered = collection.query(
        query_embeddings=query_embedding,
        n_results=5,
        where={"category": "AI"}  # Filter by metadata
    )
    
    print("Query Results (Filtered: category='AI'):")
    print("-" * 70)
    for i, (doc, metadata) in enumerate(
        zip(results_filtered['documents'][0], results_filtered['metadatas'][0]), 
        1
    ):
        print(f"\n{i}. {doc}")
        print(f"   Category: {metadata['category']}")
    
    # Compare with unfiltered results
    results_unfiltered = collection.query(
        query_embeddings=query_embedding,
        n_results=5
    )
    
    print("\n\nQuery Results (No Filter):")
    print("-" * 70)
    for i, (doc, metadata) in enumerate(
        zip(results_unfiltered['documents'][0], results_unfiltered['metadatas'][0]), 
        1
    ):
        print(f"\n{i}. {doc}")
        print(f"   Category: {metadata['category']}")
        
else:
    print("Collection or model not available")


Query Results (Filtered: category='AI'):
----------------------------------------------------------------------

1. Machine learning is a subset of artificial intelligence
   Category: AI

2. Supervised learning uses labeled data to train models
   Category: AI

3. Computer vision enables machines to interpret visual information
   Category: AI

4. Deep learning uses neural networks with multiple layers
   Category: AI

5. Unsupervised learning finds patterns in unlabeled data
   Category: AI


Query Results (No Filter):
----------------------------------------------------------------------

1. Machine learning is a subset of artificial intelligence
   Category: AI

2. Supervised learning uses labeled data to train models
   Category: AI

3. Computer vision enables machines to interpret visual information
   Category: AI

4. Deep learning uses neural networks with multiple layers
   Category: AI

5. Unsupervised learning finds patterns in unlabeled data
   Category: AI


### 3.4 ChromaDB - Persistent Storage

ChromaDB can save collections to disk for persistence across sessions.


In [12]:
try:
    # Create persistent client
    persistent_client = chromadb.PersistentClient(path="./chroma_db")
    
    # Create or get collection
    persistent_collection = persistent_client.get_or_create_collection(
        name="persistent_docs"
    )
    
    if persistent_collection.count() == 0:
        # Add documents if collection is empty
        if model:
            embeddings_list = model.encode(documents[:5]).tolist()
            persistent_collection.add(
                embeddings=embeddings_list,
                documents=documents[:5],
                ids=[f"persist_{i}" for i in range(5)]
            )
            print(f"Added {persistent_collection.count()} documents to persistent collection")
        else:
            print("Model not available")
    else:
        print(f"Loaded existing collection with {persistent_collection.count()} documents")
    
    # Query the persistent collection
    if model:
        query_embedding = model.encode(["artificial intelligence"]).tolist()
        results = persistent_collection.query(
            query_embeddings=query_embedding,
            n_results=2
        )
        
        print("\nQuery Results from Persistent Collection:")
        for i, doc in enumerate(results['documents'][0], 1):
            print(f"{i}. {doc}")
    
except Exception as e:
    print(f"Error with persistent storage: {e}")
    print("Note: Persistent storage saves data to './chroma_db' directory")


Added 5 documents to persistent collection

Query Results from Persistent Collection:
1. Machine learning is a subset of artificial intelligence
2. Natural language processing helps computers understand human language


## 4. Comparison: FAISS vs ChromaDB

### FAISS
**Pros:**
- Extremely fast and optimized
- Handles billions of vectors
- Multiple indexing algorithms
- GPU support
- Lightweight (just the search library)

**Cons:**
- Lower-level API (more code needed)
- No built-in metadata filtering
- No automatic embedding generation
- Manual persistence management

**Best for:** Large-scale similarity search, research, when you need maximum performance

### ChromaDB
**Pros:**
- Easy-to-use high-level API
- Built-in embedding generation
- Metadata filtering and querying
- Automatic persistence
- Production-ready features

**Cons:**
- Less control over indexing algorithms
- May be slower for very large datasets
- More dependencies

**Best for:** Production applications, RAG systems, when you need metadata filtering

### When to Use Which?

- **FAISS**: When you need maximum performance, have millions+ vectors, or want fine-grained control
- **ChromaDB**: When you need metadata filtering, want easier setup, or building production RAG systems


## 5. Practical Example: Building a Simple RAG System

Let's build a simple RAG (Retrieval Augmented Generation) system using ChromaDB to demonstrate a real-world use case.


In [13]:
# Simple RAG System Example
if client is not None and model is not None:
    # Create a knowledge base
    knowledge_base = [
        "Machine learning algorithms learn patterns from data without explicit programming.",
        "Deep learning uses neural networks with multiple hidden layers to learn complex patterns.",
        "Natural language processing enables computers to understand and generate human language.",
        "Computer vision allows machines to interpret and understand visual information from images.",
        "Reinforcement learning trains agents to make decisions through trial and error with rewards.",
        "Supervised learning requires labeled training data to learn input-output mappings.",
        "Unsupervised learning discovers hidden patterns in data without labels.",
        "Transfer learning allows models trained on one task to be adapted for related tasks."
    ]
    
    # Create RAG collection
    rag_collection = client.create_collection(name="rag_knowledge_base")
    
    # Add knowledge base documents
    rag_collection.add(
        documents=knowledge_base,
        ids=[f"kb_{i}" for i in range(len(knowledge_base))],
        metadatas=[{"source": "knowledge_base", "topic": "AI/ML"} for _ in knowledge_base]
    )
    
    print(f"Knowledge base created with {rag_collection.count()} documents")
    
    # Simulate a user query
    user_query = "How does deep learning work?"
    
    # Retrieve relevant context
    results = rag_collection.query(
        query_texts=[user_query],
        n_results=3
    )
    
    print(f"\nUser Query: '{user_query}'")
    print("\nRetrieved Context (for LLM):")
    print("=" * 70)
    for i, doc in enumerate(results['documents'][0], 1):
        print(f"\n{i}. {doc}")
    
    # In a real RAG system, you would:
    # 1. Retrieve relevant documents (done above)
    # 2. Combine them into context
    # 3. Send context + query to LLM
    # 4. Return LLM response
    
    context = "\n\n".join(results['documents'][0])
    print("\n\n" + "=" * 70)
    print("Context for LLM (would be sent to GPT/Claude/etc.):")
    print("=" * 70)
    print(f"\nContext:\n{context}\n\nUser Question: {user_query}")
    
else:
    print("ChromaDB or model not available")


Knowledge base created with 8 documents

User Query: 'How does deep learning work?'

Retrieved Context (for LLM):

1. Deep learning uses neural networks with multiple hidden layers to learn complex patterns.

2. Computer vision allows machines to interpret and understand visual information from images.

3. Machine learning algorithms learn patterns from data without explicit programming.


Context for LLM (would be sent to GPT/Claude/etc.):

Context:
Deep learning uses neural networks with multiple hidden layers to learn complex patterns.

Computer vision allows machines to interpret and understand visual information from images.

Machine learning algorithms learn patterns from data without explicit programming.

User Question: How does deep learning work?


## Summary

In this notebook, we've covered:

1. **What Vector DBs are**: Specialized databases for efficient similarity search
2. **FAISS**: Fast similarity search library with multiple indexing methods
3. **ChromaDB**: Easy-to-use embedding database with built-in features
4. **Practical Applications**: RAG systems, semantic search, and more

### Key Takeaways:

- Vector DBs are essential for scaling similarity search beyond thousands of vectors
- FAISS provides maximum performance and flexibility
- ChromaDB offers ease of use and production-ready features
- Both are valuable tools depending on your use case

### Next Steps:

- Explore other vector DBs (Pinecone, Weaviate, Qdrant)
- Build a complete RAG system with LLM integration
- Experiment with different embedding models
- Scale to larger datasets and measure performance


In [15]:
sample = [
  {
    "name": "John Doe",
    "emails": ["john.doe@example.com", "j.doe@work.com"]
  },
  {
    "name": "Jane Smith",
    "emails": ["jane.smith@example.com", "jsmith@business.org", "jane@home.net"]
  },
  {
    "name": "Alice Williams",
    "emails": ["alice.w@mail.com", "family.w@shared.com"]
  },
  {
    "name": "Bob Williams",
    "emails": ["bob@personal.io", "family.w@shared.com"]
  },
  {
    "name": "Charlie Williams",
    "emails": ["charlie.w@web.com", "family.w@shared.com"]
  },
  {
    "name": "David Brown",
    "emails": ["david.brown@unique.com", "d.brown@university.edu"]
  },
  {
    "name": "Johnny Doe",
    "emails": ["john.doe@example.com", "johnny@gmail.com"]
  },
  {
    "name": "J. Smith",
    "emails": ["jsmith@business.org", "jane.smith@example.com", "jsmith@business.org"]
  },
  {
    "name": "Emily Davis",
    "emails": ["emily.d@fastmail.com"]
  },
  {
    "name": "Frank Miller",
    "emails": []
  }
]

## Understanding Union-Find (Disjoint Set Union)

**Union-Find** (also called Disjoint Set Union) is a data structure that efficiently tracks groups of connected elements. It's perfect for finding connected components in graphs.

### The Problem It Solves

Imagine you have multiple items, and you want to know which items are "connected" to each other. In our case:
- Accounts are connected if they share an email
- We want to find all groups of connected accounts

### How It Works

Union-Find uses two main operations:

1. **`find(x)`**: Finds the "representative" (root) of the group containing `x`
2. **`union(x, y)`**: Merges the groups containing `x` and `y` into one group

### Visual Example

Let's say we have 5 accounts (0-4):
- Account 0 and 1 share an email → connect them
- Account 2 and 3 share an email → connect them  
- Account 1 and 2 share an email → connect them

```
Initially: [0] [1] [2] [3] [4]  (5 separate groups)

After union(0, 1): [0-1] [2] [3] [4]  (4 groups)

After union(2, 3): [0-1] [2-3] [4]  (3 groups)

After union(1, 2): [0-1-2-3] [4]  (2 groups - all connected!)
```

### Implementation Details

- **Parent Array**: Each element points to its parent. The root points to itself.
- **Path Compression**: When finding the root, we update all nodes along the path to point directly to the root (makes future lookups faster)
- **Union by Rank**: (Optional optimization) Always attach smaller tree to larger tree

### Why Use It Here?

For account merging:
- If Account A shares email with Account B, and Account B shares email with Account C
- Then A, B, and C should all be in the same merged group
- Union-Find automatically handles these transitive connections!


In [ ]:
# Step-by-step Union-Find demonstration
# Let's trace through a simple example

print("=" * 70)
print("Union-Find Step-by-Step Example")
print("=" * 70)

# Simple example: 4 accounts
accounts_simple = [
    {"name": "A", "emails": ["email1"]},
    {"name": "B", "emails": ["email1", "email2"]},  # Shares email1 with A
    {"name": "C", "emails": ["email2", "email3"]},  # Shares email2 with B
    {"name": "D", "emails": ["email4"]}  # Isolated
]

n = len(accounts_simple)
parent = list(range(n))  # Initially, each account is its own parent

def find(x):
    """Find the root of x with path compression"""
    if parent[x] != x:
        parent[x] = find(parent[x])  # Path compression
    return parent[x]

def union(x, y):
    """Union the groups containing x and y"""
    root_x = find(x)
    root_y = find(y)
    if root_x != root_y:
        parent[root_y] = root_x
        return True  # Actually merged
    return False  # Already in same group

print(f"\nInitial state: {parent}")
print("Each account is its own parent (separate groups)\n")

# Build email mapping
from collections import defaultdict
email_to_accounts = defaultdict(set)
for idx, account in enumerate(accounts_simple):
    for email in account["emails"]:
        email_to_accounts[email].add(idx)

print("Email to accounts mapping:")
for email, indices in email_to_accounts.items():
    print(f"  {email}: accounts {sorted(indices)}")

print("\n" + "-" * 70)
print("Performing unions for accounts sharing emails:")
print("-" * 70)

step = 1
for email, account_indices in email_to_accounts.items():
    account_list = list(account_indices)
    if len(account_list) > 1:
        print(f"\nStep {step}: Email '{email}' connects accounts {account_list}")
        for i in range(1, len(account_list)):
            merged = union(account_list[0], account_list[i])
            if merged:
                print(f"  → Union({account_list[0]}, {account_list[i]})")
                print(f"  → Parent array: {parent}")
                print(f"  → Groups: ", end="")
                # Show current groups
                groups = defaultdict(list)
                for idx in range(n):
                    groups[find(idx)].append(idx)
                print([sorted(groups[g]) for g in groups])
        step += 1

print("\n" + "=" * 70)
print("Final Result:")
print("=" * 70)
print(f"Parent array: {parent}")

# Show final groups
final_groups = defaultdict(list)
for idx in range(n):
    root = find(idx)
    final_groups[root].append(idx)

print("\nMerged groups:")
for root, indices in final_groups.items():
    names = [accounts_simple[i]["name"] for i in indices]
    emails = set()
    for i in indices:
        emails.update(accounts_simple[i]["emails"])
    print(f"  Group {root}: {names} → {sorted(emails)}")

print("\n" + "=" * 70)
print("Key Insight:")
print("=" * 70)
print("Account A and B share email1, B and C share email2.")
print("Even though A and C don't directly share an email,")
print("Union-Find connects them through B! This is called 'transitive closure'.")


In [ ]:
# Merge accounts that share the same email addresses
# Using a graph-based approach to find connected components

def merge_accounts_by_email(accounts):
    """
    Merge accounts that share at least one email address.
    Returns a list of merged accounts with names_used and emails.
    """
    from collections import defaultdict
    
    # Build email-to-account mapping
    email_to_accounts = defaultdict(set)
    for idx, account in enumerate(accounts):
        for email in account.get("emails", []):
            if email:  # Skip empty emails
                email_to_accounts[email].add(idx)
    
    # Union-Find (Disjoint Set) to find connected components
    parent = list(range(len(accounts)))
    
    def find(x):
        if parent[x] != x:
            parent[x] = find(parent[x])  # Path compression
        return parent[x]
    
    def union(x, y):
        root_x = find(x)
        root_y = find(y)
        if root_x != root_y:
            parent[root_y] = root_x
    
    # Union accounts that share emails
    for email, account_indices in email_to_accounts.items():
        account_list = list(account_indices)
        for i in range(1, len(account_list)):
            union(account_list[0], account_list[i])
    
    # Group accounts by their root
    merged_groups = defaultdict(lambda: {"names_used": [], "emails": set()})
    
    for idx, account in enumerate(accounts):
        root = find(idx)
        merged_groups[root]["names_used"].append(account["name"])
        # Add all emails from this account
        for email in account.get("emails", []):
            if email:  # Skip empty emails
                merged_groups[root]["emails"].add(email)
    
    # Convert to final format
    merged_accounts = []
    for group in merged_groups.values():
        merged_accounts.append({
            "names_used": sorted(group["names_used"]),  # Sort for consistency
            "emails": sorted(list(group["emails"]))  # Sort for consistency
        })
    
    return merged_accounts

# Merge the sample accounts
merged = merge_accounts_by_email(sample)

print("Original accounts:", len(sample))
print("Merged accounts:", len(merged))
print("\n" + "="*70)
print("Merged Accounts:")
print("="*70)

for i, account in enumerate(merged, 1):
    print(f"\n{i}. Names: {account['names_used']}")
    print(f"   Emails: {account['emails']}")


defaultdict(<class 'set'>, {'john.doe@example.com': {0, 6}, 'j.doe@work.com': {0}, 'jane.smith@example.com': {1, 7}, 'jsmith@business.org': {1, 7}, 'jane@home.net': {1}, 'alice.w@mail.com': {2}, 'family.w@shared.com': {2, 3, 4}, 'bob@personal.io': {3}, 'charlie.w@web.com': {4}, 'david.brown@unique.com': {5}, 'd.brown@university.edu': {5}, 'johnny@gmail.com': {6}, 'emily.d@fastmail.com': {8}})
Original accounts: 10
Merged accounts: 6

Merged Accounts:

1. Names: ['John Doe', 'Johnny Doe']
   Emails: ['j.doe@work.com', 'john.doe@example.com', 'johnny@gmail.com']

2. Names: ['J. Smith', 'Jane Smith']
   Emails: ['jane.smith@example.com', 'jane@home.net', 'jsmith@business.org']

3. Names: ['Alice Williams', 'Bob Williams', 'Charlie Williams']
   Emails: ['alice.w@mail.com', 'bob@personal.io', 'charlie.w@web.com', 'family.w@shared.com']

4. Names: ['David Brown']
   Emails: ['d.brown@university.edu', 'david.brown@unique.com']

5. Names: ['Emily Davis']
   Emails: ['emily.d@fastmail.com']

6